In [ ]:
!pip install pip --upgrade
!pip install tweepy --upgrade
!pip install Sastrawi
!pip install numpy
!pip install sklearn
!pip install textblob
!pip install pandas
!pip install matplotlib
!pip install nltk==3.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [ ]:
import tweepy

BEARERTOKEN = 'AAAAAAAAAAAAAAAAAAAAAJQrYQEAAAAA1BtlexYIrHRZvqkz6TEOraEC%2B0k%3D5cOp6U7ytEMpcTDctDQWVl6HFd0Y28pxhDnTGyHA9KiiC1MVCF'

client = tweepy.Client(bearer_token=BEARERTOKEN)

query = '"Tentang Ruang Guru" OR "Ruang Guru" OR "menurut saya Ruang Guru" lang:en'

# get data max 100
responses = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at'])
tweet100 = [tweet for tweet in responses.data]

# get max 1000
tweettexts = [tweet.text for tweet in tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100).flatten(limit=1000)]

In [ ]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
import re

In [ ]:
data = {
    'text' : tweettexts
}
df = pd.DataFrame(data)
df.to_csv(f"crawling.csv", mode='w', index=False, header=data.keys())

In [ ]:
import pandas as pd

df = pd.read_csv('crawling.csv', encoding='unicode_escape')
df.head(5)

,text
0,Mau ngomong shibal versi indo di ruang guru sa...
1,Permisi @ruangguru ini emang kalo abis daftar ...
2,RT @scubepid: heehoon oneshot ð\n\n; seanda...
3,"langganannya sekali, diterror nya brpuluh"" kal..."
4,"kalo sekolah biasa mah kaga papa sendirian, in..."


In [ ]:
#Cleaning Text
def textcleaner(text):
    new_text = str(text)
    
    # remove old style retweet text "RT"
    new_text = re.sub(r'^RT\s+', '', new_text)
    
    # remove username
    new_text = re.sub(r'@([A-Za-z0-9_]+)', '', new_text)

    # remove hyperlinks
    new_text = re.sub(r'https?:\/\/.*[\r\n]*', '', new_text)

    # remove invalid character
    new_text = re.sub("[^A-Za-z" "]+", " ", new_text)
    
    # menghilangkan 2 kata
    new_text    = re.sub(r'\b\w(1,2)\b',' ', new_text)
    
    return new_text

In [ ]:
def caselower(sentences):
    return sentences.lower()

In [ ]:
def tokenmaker(sentences):
    return sentences.strip().split()

In [ ]:
#Stop Removal
from nltk.corpus import stopwords
nltk.download('stopwords')

stopword = set(stopwords.words('indonesian'))

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

In [ ]:
# create labeler

pos_kata = [pos.strip() for pos in open('kata_positif.txt', 'r').readlines()]
neg_kata = [neg.strip() for neg in open('kata_negatif.txt', 'r').readlines()]

#list kata-kata negasi
negasi = [neg.strip() for neg in open('negasi.txt', 'r').readlines()]


def labeling(stemmed):
    count_pos = 0
    count_neg = 0
    for ind, stem in enumerate(stemmed):
        if stem.strip() in pos_kata:
            if stemmed[ind-1].strip() in negasi: #untuk cek kata sebelumnya 
                count_neg += 1
            else:
                count_pos += 1
        elif stem.strip() in neg_kata:
            if stemmed[ind-1].strip() in negasi:
                count_pos += 1
            else:
                count_neg += 1

    return 'NETRAL' if count_pos == count_neg else 'POSITIF' if count_pos > count_neg else 'NEGATIF'
    


In [ ]:
#@title
kalimat     = "menurut saya dia sangat dia baik, tapi terkadang jahat" # label positif

kata = [textcleaner(kata).strip() for kata in kalimat.split()]
print(kata)
kata_positif = ['baik', 'bagus', 'keren']
kata_negatif = ['buruk', 'salah', 'jahat']

negasi = [negasi.strip() for negasi in open('negasi.txt', 'r').readlines()]

neg = 0
pos = 0
for i, kt in enumerate(kata):
  # cek apakah kata tersebut ada pada kata positif
  # print(f'kata {kt} index {i}')
  if kt in kata_positif:
    # ada ngga kata negasi sebelumnya
    # print(f'kata {kt} termasuk kata positif')
    if kata[i-1] in negasi:
      # print(f'kata {kt} dinegasikan')
      neg += 1
    else:
      pos += 1
  elif kt in kata_negatif:
    if kata[i-1] in negasi:
      pos += 1
    else:
      neg += 1

print(f'positif = {pos}')
print(f'negatif = {neg}')

hasil = ""
if pos == neg:
  hasil = "netral"
elif pos > neg:
  hasil = "positif"
else:
  hasil = "negatif"

print(hasil)
  

['menurut', 'saya', 'dia', 'sangat', 'dia', 'baik', 'tapi', 'terkadang', 'jahat']
positif = 1
negatif = 1
netral


In [ ]:
# cleaning text
df['clean_text'] = df['text'].apply(lambda x : textcleaner(x))

In [ ]:
# convert ke lower case
df['lower'] = df['clean_text'].apply(lambda x : caselower(x))

In [ ]:
# tokenizing
df['token'] = df['lower'].apply(lambda x : tokenmaker(x))

In [ ]:
# stopword removing
df["stop_removed"] = df['token'].apply(lambda x: remove_stopwords(x))

In [ ]:
# stemming
df["stemmed"] = df['stop_removed'].apply(lambda x: stemming(x))
print("Stemming selesai!")

Stemming selesai!


In [ ]:
# labelling sentiment
df["label"] = df['stemmed'].apply(lambda x: labeling(x))
df.to_csv(f"labeled.csv", mode='w', index=False, header=True)
df.head(df.size)

,text,clean_text,lower,token,stop_removed,stemmed,label
0,Mau ngomong shibal versi indo di ruang guru sa...,Mau ngomong shibal versi indo di ruang guru sa...,mau ngomong shibal versi indo di ruang guru sa...,"[mau, ngomong, shibal, versi, indo, di, ruang,...","[ngomong, shibal, versi, indo, ruang, guru, tr...","[ngomong, shibal, versi, indo, ruang, guru, tr...",NEGATIF
1,Permisi @ruangguru ini emang kalo abis daftar ...,Permisi ini emang kalo abis daftar ruang guru ...,permisi ini emang kalo abis daftar ruang guru ...,"[permisi, ini, emang, kalo, abis, daftar, ruan...","[permisi, emang, kalo, abis, daftar, ruang, gu...","[permisi, emang, kalo, abis, daftar, ruang, gu...",NEGATIF
2,RT @scubepid: heehoon oneshot ð\n\n; seanda...,heehoon oneshot seandainya aku tidak lama unt...,heehoon oneshot seandainya aku tidak lama unt...,"[heehoon, oneshot, seandainya, aku, tidak, lam...","[heehoon, oneshot, seandainya, mengambil, remo...","[heehoon, oneshot, anda, ambil, remot, ac, rua...",NETRAL
3,"langganannya sekali, diterror nya brpuluh"" kal...",langganannya sekali diterror nya brpuluh kali ...,langganannya sekali diterror nya brpuluh kali ...,"[langganannya, sekali, diterror, nya, brpuluh,...","[langganannya, diterror, nya, brpuluh, kali, s...","[langgan, diterror, nya, brpuluh, kali, sp, kl...",NETRAL
4,"kalo sekolah biasa mah kaga papa sendirian, in...",kalo sekolah biasa mah kaga papa sendirian ini...,kalo sekolah biasa mah kaga papa sendirian ini...,"[kalo, sekolah, biasa, mah, kaga, papa, sendir...","[kalo, sekolah, mah, kaga, papa, sekolah, ngum...","[kalo, sekolah, mah, kaga, papa, sekolah, ngum...",NETRAL
...,...,...,...,...,...,...,...
862,"Rehab Ruang Kelas, Ruang Guru, Pembangunan Rua...",Rehab Ruang Kelas Ruang Guru Pembangunan Ruang...,rehab ruang kelas ruang guru pembangunan ruang...,"[rehab, ruang, kelas, ruang, guru, pembangunan...","[rehab, ruang, kelas, ruang, guru, pembangunan...","[rehab, ruang, kelas, ruang, guru, bangun, rua...",POSITIF
863,@cloudyskev Waktu itu yg br1ght di ruang guru ...,Waktu itu yg br ght di ruang guru aja begitu ...,waktu itu yg br ght di ruang guru aja begitu ...,"[waktu, itu, yg, br, ght, di, ruang, guru, aja...","[yg, br, ght, ruang, guru, aja, reaksinya, ora...","[yg, br, ght, ruang, guru, aja, reaksi, orang,...",NETRAL
864,RT @rarabyjane: Sponsornya ruang guru jsjsjs n...,Sponsornya ruang guru jsjsjs nanti jaemin ket...,sponsornya ruang guru jsjsjs nanti jaemin ket...,"[sponsornya, ruang, guru, jsjsjs, nanti, jaemi...","[sponsornya, ruang, guru, jsjsjs, jaemin, kete...","[sponsor, ruang, guru, jsjsjs, jaemin, ketemu,...",NETRAL
865,RT @_naramustika: Rangkuman TKA Biologi UTBK\n...,Rangkuman TKA Biologi UTBK Sumber bimbel zeni...,rangkuman tka biologi utbk sumber bimbel zeni...,"[rangkuman, tka, biologi, utbk, sumber, bimbel...","[rangkuman, tka, biologi, utbk, sumber, bimbel...","[rangkum, tka, biologi, utbk, sumber, bimbel, ...",NETRAL


In [ ]:
# convert label to polarity
def convert(polarity):
    if polarity == 'POSITIF':
        return 1
    elif polarity == 'NETRAL':
        return 0
    else:
        return -1

In [ ]:
df = pd.read_csv('labeled.csv')
df['polarity'] = df['label'].apply(lambda x: convert(x))
df.head(5)

,text,clean_text,lower,token,stop_removed,stemmed,label,polarity
0,Mau ngomong shibal versi indo di ruang guru sa...,Mau ngomong shibal versi indo di ruang guru sa...,mau ngomong shibal versi indo di ruang guru sa...,"['mau', 'ngomong', 'shibal', 'versi', 'indo', ...","['ngomong', 'shibal', 'versi', 'indo', 'ruang'...","['ngomong', 'shibal', 'versi', 'indo', 'ruang'...",NEGATIF,-1
1,Permisi @ruangguru ini emang kalo abis daftar ...,Permisi ini emang kalo abis daftar ruang guru ...,permisi ini emang kalo abis daftar ruang guru ...,"['permisi', 'ini', 'emang', 'kalo', 'abis', 'd...","['permisi', 'emang', 'kalo', 'abis', 'daftar',...","['permisi', 'emang', 'kalo', 'abis', 'daftar',...",NEGATIF,-1
2,RT @scubepid: heehoon oneshot ð\n\n; seanda...,heehoon oneshot seandainya aku tidak lama unt...,heehoon oneshot seandainya aku tidak lama unt...,"['heehoon', 'oneshot', 'seandainya', 'aku', 't...","['heehoon', 'oneshot', 'seandainya', 'mengambi...","['heehoon', 'oneshot', 'anda', 'ambil', 'remot...",NETRAL,0
3,"langganannya sekali, diterror nya brpuluh"" kal...",langganannya sekali diterror nya brpuluh kali ...,langganannya sekali diterror nya brpuluh kali ...,"['langganannya', 'sekali', 'diterror', 'nya', ...","['langganannya', 'diterror', 'nya', 'brpuluh',...","['langgan', 'diterror', 'nya', 'brpuluh', 'kal...",NETRAL,0
4,"kalo sekolah biasa mah kaga papa sendirian, in...",kalo sekolah biasa mah kaga papa sendirian ini...,kalo sekolah biasa mah kaga papa sendirian ini...,"['kalo', 'sekolah', 'biasa', 'mah', 'kaga', 'p...","['kalo', 'sekolah', 'mah', 'kaga', 'papa', 'se...","['kalo', 'sekolah', 'mah', 'kaga', 'papa', 'se...",NETRAL,0


In [ ]:
x = df['stop_removed']
y = df['polarity']